# Programação Quadrática

O objetivo é utilizar um Solver de código-fonte aberto, bem como mostrar a configuração programática geral, para lidar com o trade-off de média / variância com as seguintes requisitos de otimização de portfólio presentes em aplicativos do mundo real:
- Objetivos de exposição bruta e líquida
- Limiares de exposição por fatores / setores
- Limites de exposição subjacentes individuais
- Restrição de cardinalidade no número de ativos no portfólio otimizado

Os três primeiros são comumente encontrados nos Solvers convexos contínuos padrão como cvxopt. O último é mais dificil de encontrar. Adicionar uma restrição de cardinalidade transforma o problema numa programação quadrática inteira mista (comum em Solvers comerciais como ibm-cplex, gurobi e matlab), onde as ferramentas de código aberto são menos acessíveis.

Seções:
- Conceitos
- Descrição do problema
- Exemplo prático

## Otimização de Portfólio

A otimização do portfólio, desde que Markowitz popularizou a fronteira eficiente, concentrou-se na relação de "perde-e-ganha" entre risco e retorno. A forma mais básica de otimização se concentra em encontrar os pesos do portfólio que maximizem o retorno considerando risco máximo ou minimizem o risco considerando um retorno mínimo. É possível encontrar um portfólio com indice Sharpe ótimo iterando os níveis de uma dessas permutações e encontrando o resultado com a maior taxa de $\frac{retorno}{risco}$, ou Sharpe.

### Pressupostos de entrada:
- Retorno (diário)
- Risco (diário)

### Restrições de entrada:
- Limites superior e inferior nas alocações
- Limites superior e inferior na alocação longa agregada (mostrado: 0,75, 1.)
- Ponderação (rho) entre a importância do retorno desejado e o risco tolerado

In [9]:
#include <iostream>

#include "csv.h"
#include "setup.h"

#include "ortools/linear_solver/linear_solver.h"
#include "ortools/linear_solver/linear_solver.pb.h"

using namespace operations_research;

O modelo padrão recai numa otimização quadrática convexa na forma:

Minimizar: $x^T\sum x$

Sujeito a: 

$
\begin{align}
p^Tx \ge r_{min} \\
1^Tx \le 1  \\
x \ge 0
\end{align}
$

Maximizar: $p^Tx - \rho x^T\sum x$

Sujeito a: 

$
\begin{align}
1^Tx \le 1  \\
x \ge 0
\end{align}
$

In [9]:
{
    // max and min portfolio allocation
    auto net_upper_long_bound=1.0;
    auto net_lower_long_bound=0.5;
    // risk/reward tradeoff input, higher means less risk, b/t .1 & 1
    auto rho = 0.5;
    
    std::ifstream f1("portfolio_desc.csv");
    auto tickers = csv_read_string(f1);
    
    std::ifstream f2("portfolio_med.csv");
    auto returns = csv_read_double(f2);
    
    std::ifstream f3("portfolio_cov.csv");
    auto covariance = csv_read_double(f3);
    
    auto n = tickers.size();
    
    MPModelRequest model_request;
    
    MPSolutionResponse solution_response;

    model_request.set_solver_type(MPModelRequest::SCIP_MIXED_INTEGER_PROGRAMMING);
    
    MPModelProto *model_proto = model_request.mutable_model();

    model_proto->set_name("Portfolio Otimization");
    model_proto->set_maximize(true);
   
    std::vector<MPVariableProto*> allocation(n);
    for(auto i = 0; i < n; i++) {
        allocation[i] = model_proto->add_variable();
        allocation[i]->set_name(tickers[i][0]);  
        allocation[i]->set_upper_bound(returns[i][1]);  
        allocation[i]->set_lower_bound(0);//long only
        allocation[i]->set_objective_coefficient(returns[i][0]); //FO: sum(return[i] * allocation[i])
        allocation[i]->set_is_integer(false);      
    }
    
    //FO: - rho * sum(covariance[i][j] * allocation[i] * allocation[j])
    MPQuadraticObjective *quad_obj = model_proto->mutable_quadratic_objective();
    for(auto i = 0; i < n; i++) {
        for(auto j = 0; j < n; j++) {
            quad_obj->add_qvar1_index(i);
            quad_obj->add_qvar2_index(j);
            quad_obj->add_coefficient(-rho*covariance[i][j]);
        }
    }
    
    //Constraint: net_lower_long_bound <= sum(allocation[i]) <= net_upper_long_bound
    MPConstraintProto* c = model_proto->add_constraint();
    c->set_name("c");  
    c->set_lower_bound(net_lower_long_bound);    
    c->set_upper_bound(net_upper_long_bound);
    for(auto i = 0; i < n; i++) {
        c->add_var_index(i);
        c->add_coefficient(1);
    }
    
    MPSolver::SolveWithProto(model_request, &solution_response);

    // The problem has an optimal solution.
    if (solution_response.status() == MPSOLVER_OPTIMAL)
        std::cout << "Optimal" << std::endl;

    std::cout << "objective = " << solution_response.objective_value() << std::endl;
    for (int i = 0; i < n; i++) {
        std::cout << model_proto->variable(i).name() << " = "
                  << solution_response.variable_value(i) << std::endl;
    }
}

Optimal
objective = 0.00154731
AGN = 0
AGNC = 0
AGO = 0.199433
ALK = 0.000566758
APOL = 0
BAH = 0
BCR = 0
BR = 0
BRCD = 0
CCK = 0
CPN = 0
AAL = 0
ALLE = 0
ALTR = 0.2
AMCX = 0
AMH = 0.2
AMT = 0
ARCP = 0.2
AWAY = 0
BMRN = 0
CAB = 0.2
